# 项目模块化

cython无法自己作为程序入口,要么用python写程序入口,要么用C写程序入口,因此cython只能用于写模块,无论是在桥接python时还是在桥接C时都是如此.



在介绍项目模块化之前,先要分清楚是在什么环境下的模块化.cython的模块化按环境可以分为3种:

1. 作为cython模块的模块化
2. 作为python模块的模块化
3. 作为C模块的模块化

这三者有共同点也有区别,下面开始分别介绍这三种环境下的模块化异同.

本文的演示代码在[]()

## 作为cython模块的模块化

作为cython模块,它的主要作用是让cython代码间可以模块化编程,从而实现抽象和复用,是配合`cimport`语法的的代码组织工具.比如我们可以写一个cython模块A,当在一个新的cython项目中需要用到A中定义过的C函数或扩展类型中定义的C方法时我们就可以用`cimport`语法导入这个模块并使用了.因此cython模块可以看做是作为python模块的模块化和作为C模块的模块化的前提,后两者是前者的应用.只有弄明白如何作为cython模块进行模块化才能在后两种环境下进行的下去.而cython作为一个python和C之间的桥接语言,cython模块是无法脱离python或C使用的,也就不存在编译一说.

cython模块主要是看声明文件`.pxd`,和python的模块系统非常类似,一个cython项目如果要作为cython模块必须满足两种形式中的一种:

+ 简单模块,可以是一个单独的`.pxd`
+ 复杂模块,可以是一个带`__init__.pxd`的文件夹.

cython的复杂模块允许内部继续包含子模块,以我们的例子`mymath`为例:

```txt
mymath---\
    |---__init__.pxd
    |---normalize_and_l2norm.pyx # 示例内部调用
    |---normalize_and_l2norm.pxd # 示例内部调用的声明
    |---inner---\
    |       |---__init__.pxd
    |       |---l2norm.pxd   # 有接口`l2norm`
    |       |---l2norm.pyx
    |
    |---median_along_axis0.pxd  # 有接口`_median_along_axis0`
    |---median_along_axis0.pyx
    |---normalize.pxd  # 有接口`_normalize`
    |---normalize.pyx
    |---notexist.pxd #示例有声明无实现
    ...
cythoncallmymath.pyx # 示例外部调用
cythoncallmymath.py # 示例外部调用纯净模式
```

### 内部调用

在其中的子模块可以和python中一样,使用相对引用导入需要的接口,如果是纯净模式则还是只能老老实实绝对引用导入,比如在一个新实现文件`mymath.pyx`中要导入接口`l2norm`和`_normalize`:

+ `normalize_and_l2norm.pyx`(cython语法)

```cython
...
from .inner.l2norm cimport l2norm
from .normalize cimport _normalize
...
```

+ `normalize_and_l2norm.py`(纯净模式语法)

```python
from cython.cimports.mymath.inner.l2norm import l2norm
from cython.cimports.mymath.normalize import _normalize
```

当然了Cython中同样要避免钻石引用.

### 外部调用

对于外部调用,无论哪种语法都只能老老实实绝对调用

+ `cythoncallmymath.pyx`(cython语法)

    ```cython
    from mymath.normalize cimport _normalize
    import numpy as np


    def callmymath():
        cdef double[:] output = _normalize(np.array([1.1,2.2,3.3,4.4]))
        print(np.asarray(output))
    ```

+ `cythoncallmymath.py`(纯净模式语法)

```cython
import cython
from cython.cimports.mymath.normalize cimport _normalize
import numpy as np


def callmymath():
    output: cython.double[:] = _normalize(np.array([1.1,2.2,3.3,4.4]))
    print(np.asarray(output))
```

## 作为python模块的模块化

模块化的cython代码需要经由`cythonize`编译为动态链接库后即可被python解释器导入使用.使用`cythonize`编译cython模块本质上就是将其中的实现部分每个文件编译为对应的同名动态链接库.也就是说一般一个座位python模块的cython模块.

### 作为Python模块的编译

`Cythonize`编译出的是python可以调用的动态链接库模块,必须指定实现部分的源文件.无论是cython的简单模块还是复杂模块,编译后都是每个实现部分源文件都会编译出一个python可以调用的动态链接库文件.因此可以说cython模块的编译本质上是将其中的所有实现部分源文件分别编译为python可以调用的动态链接库文件.

`cythonize`支持指定的资源可以是单一的cython实现部分源码也可以是复杂模块.
如果我们编译的是个单文件的模块,那么没什么好纠结的,比如上面的例子中的`cythoncallmymath.pyx`,编译直接使用命令`cythonize -i --3 cythoncallmymath.pyx`即可,使用时也是直接`import cythoncallmymath`即可;而编译复杂模块比如上面的`mymath`则可以直接使用`cythonize -i --3 mymath`进行编译,使用`import mymath`的方式导入.但这并不意味着这种方式就是唯一方式,实际上对于编译复杂模块有3种选择:

+ 不带`__init__.py`或`__init__.pyx`.使用`cythonize -i --3 mymath`进行编译.编译好后会长这样(以macos下为例):

    ```txt
    mymath---\
        |---__init__.pxd
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    python正常模块用`__init__.py`识别,没有它就会作为[命名空间包](https://blog.hszofficial.site/TutorialForPython/#/%E8%AF%AD%E6%B3%95%E7%AF%87/%E6%A8%A1%E5%9D%97/%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4%E5%8C%85)处理.因此`mymath`模块这种方式下就只能作为命名空间包使用了.这虽然不影响使用,但多少会影响加载速度.

+ 带`__init__.py`或`__init__.pyx`.使用`cythonize -i --3 mymath`进行编译.编译好后会长这样(以macos下为例):

    ```txt
    mymath---\
        |---__init__.pxd
        |---__init__.pyx
        |---__init__.cpp
        |---__init__.cpython-310-darwin.so # 实际有用的
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---__init__.pyx
        |       |---__init__.cpp
        |       |---__init__.cpython-310-darwin.so # 实际有用的
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    因为有`__init__.cpython-310-darwin.so`存在,`mymath`模块这种方式下可以作为正常模块使用.但由于编译,原本只要几百个字节的`__init__`文件会被编译为一个100多k的动态连接库.这就有略有点浪费了.当然你也可以之后删掉换成同功能的`__init__.py`,不过如果模块构造复杂这就太麻烦了.

+ 将复杂模块作为简单模块进行编译--我们在构造模块时让模块中只有`__init__.py`是`.py`文件,其他的源文件都是`.pyx`文件,编译时指定source不再指定模块目录而是使用通配符`*`查找其中的所有`.pyx`文件进行编译,用上面的例子就是`cythonize -i --3 mymath/**/*.pyx`.编译好后会长这样(以macos下为例):
    ```txt
    mymath---\
        |---__init__.pxd
        |---__init__.py
        |---normalize_and_l2norm.pyx
        |---normalize_and_l2norm.pxd
        |---normalize_and_l2norm.cpp
        |---normalize_and_l2norm.cpython-310-darwin.so # 实际有用的
        |---inner---\
        |       |---__init__.pxd
        |       |---__init__.py
        |       |---l2norm.pxd
        |       |---l2norm.pyx
        |       |---l2norm.cpp
        |       |---l2norm.cpython-310-darwin.so  # 实际有用的
        |---median_along_axis0.pxd
        |---median_along_axis0.pyx
        |---median_along_axis0.cpp
        |---median_along_axis0.cpython-310-darwin.so  # 实际有用的
        |---normalize.pxd
        |---normalize.pyx
        |---normalize.cpp
        |---normalize.cpython-310-darwin.so  # 实际有用的
        |---notexist.pxd #示例有声明无实现
    ```

    这种方式编译好后我们的模块即可当做正常模块使用,又没有因为编译额外的`__init__`.个人认为这种方式是最优雅的.

### 没有实现的接口

cython的模块系统仅仅只是解决代码组织问题,无论哪种形式,光有声明文件也是没用的.使用`cimport`导入cython模块必须要有对应的实现部分,这个实现部分可以是模块中源文件`Cythonize`编译后的动态链接库,也可以是外部C/C++代码或库.如果没有,在运行时导入模块时一样会报`ModuleNotFoundError`错.


## 作为C模块的模块化

C/C++中模块化使用的是头文件和`#include`语法,这意味着如果我们希望在C/C++中调用cython模块则需要让cython模块提供头文件.

在Cython中有两种方式声明出来的接口可以提供头文件

+ `public`关键字
+ `api`关键字

这两个关键字都需要放在`cdef`后面.

除了要提供接口的头文件外,我们还需要有一个入口的C源码文件,然后使用编译器对所有的源码进行编译.

总结Cython作为C模块的步骤可以分为如下几步:

1. 设置`public`/`api`关键字指定接口头文件
2. 使用命令行工具`cython`转译cython源码为C/C++源码和头文件
3. [可选]提供入口/测试源码,如果是编译为链接库就没必要
4. 使用gcc/clang等C/C++编译器编译

其中在Cython中可以决定的步骤在1,2两步.

### public关键字

public关键字仅可以用于修饰定义C中有的结构声明,也就是说不能用于扩展类型或python可以直接调用的函数.

```cython
cdef public struct Bunny:  # struct声明
    int vorpalness

cdef public int spam  # 变量声明

cdef public void grail(Bunny *)  # 函数声明
```

仅有`public`关键字的情况下会在转译步骤生成以一个和声明文件同名的头文件,且被声明为`public`的接口就会在这个文件中.在C/C++中需要使用这些接口时使用`#include`语法导入即可.

在C代码中对其进行调用也就是和调用C中对应的对象一样,


In [1]:
%load_ext cython

In [14]:
%%cython -c=-Wno-unreachable-code
# distutils: language = c++

from libcpp.string cimport string, to_string
import time

cdef float get_nowtime():
    cdef float now = time.time()
    return now

cdef void hello(char* name) except *:
    cdef string ts = to_string(get_nowtime())
    cdef string out = b"hello "
    out.append(name)
    out.append(b" now timestamp is ")
    out.append(ts)
    print(out.decode("utf-8"))
    
hello(b"abc")

Content of stderr:
ld: warning: dylib (/usr/local/Cellar/gcc/13.1.0/lib/gcc/current/libstdc++.dylib) was built for newer macOS version (11.0) than being linked (10.9)hello abc now timestamp is 1700220416.000000


> 例子: 

### api关键字

api关键字和public关键字非常类似,仅可以用于修饰定义C中有的结构声明,它的不同之处在于它会创建一个以`{模块名}_api.h`为名的头文件,其中

modulename_api.h的头文件，其中包含函数和扩展类型的声明，以及一个称为import_modulename（）的函数。

我们所有需要导出外部可以访问的函数的参数类型会返回值类型都需要设置为`public`,
Cython模块中定义的任何类型，用作导出函数的参数或返回类型，都需要声明为公共类型，否则它们将不会包含在生成的头文件中，并且当您试图编译使用该头的C文件时，会出现错误